In [0]:
def criar_view_temporaria(nome_tabela):
    # Construindo o caminho do arquivo CSV com base no nome da tabela
    caminho_csv = f"/Volumes/workspace/bronze/autoseg/{nome_tabela}.csv"
    
    # Leitura do arquivo CSV usando PySpark
    df = spark.read.format("csv") \
        .option("header", "true") \
        .option("inferSchema", "true") \
        .option("sep", ';') \
        .load(caminho_csv)
    
    # Criando uma view temporária com o nome especificado
    df.createOrReplaceTempView(nome_tabela)
    
    # Exibindo as primeiras 10 linhas da view temporária
    print(f"\nTabela: {nome_tabela}")
    spark.sql(f"SELECT * FROM {nome_tabela} LIMIT 10").display()

criar_view_temporaria(nome_tabela="auto_cau")
criar_view_temporaria(nome_tabela="auto_cat")
criar_view_temporaria(nome_tabela="auto_cep")
criar_view_temporaria(nome_tabela="auto_cidade")
criar_view_temporaria(nome_tabela="auto_cob")
criar_view_temporaria(nome_tabela="auto_idade")
criar_view_temporaria(nome_tabela="auto_reg")
criar_view_temporaria(nome_tabela="auto_sexo")
criar_view_temporaria(nome_tabela="auto2_grupo")
criar_view_temporaria(nome_tabela="auto2_vei")
criar_view_temporaria(nome_tabela="PremReg")
criar_view_temporaria(nome_tabela="arq_casco_comp")
criar_view_temporaria(nome_tabela="arq_casco3_comp")
criar_view_temporaria(nome_tabela="arq_casco4_comp")
criar_view_temporaria(nome_tabela="SinReg")


Questão: Após realizar as análises pedidas (idade, prêmio, IS_MEDIA), como você correlacionaria esses resultados para criar um perfil detalhado dos segurados por **região, sexo e ano do modelo**? Quais insights estratégicos você pode extrair desses dados?

Criei um perfil detalhado de pessoas físicas com idade informada por região, sexo, idade, média do prêmio e quantidade de segurados nessas condições.
Um dos insight extratégico que percebo é que: A maior quantidade de segurados se encontram em São Paulo (Principalmente região metropolitana) e são homens, há uma grande distância até que o primeiro grupo de mulheres apareça na projeção. Mulheres são 58% dos compradores de veículos no Brasil e a maior parte dos segurados são homens, segundo esses dados. Então é evidente que há uma clara necessidade e oportunidade de crescimento no mercado de seguros no público feminino.

In [0]:
%sql
SELECT ar.DESCRICAO, acc.SEXO, acc.IDADE, ROUND(AVG(try_cast(acc.premio1 as FLOAT)), 2) AS PREMIO_MEDIA, COUNT(*) AS QNT
FROM arq_casco_comp acc
INNER JOIN
auto_reg ar
ON try_cast(acc.REGIAO as INT) = ar.CODIGO
WHERE (SEXO = 'M' OR SEXO = 'F') AND IDADE != 0
GROUP BY DESCRICAO, SEXO, IDADE
ORDER BY 5 DESC;

Questão: Qual seria a consulta SQL para descobrir a média de sinistros (numSinistros) por região no arquivo SinReg? O que esses dados revelam sobre a frequência de sinistros em diferentes regiões?

Os dados mostram o esperado: Regiões metropolitanas, devido ao alto fluxo de veículos, tem a maior frequência de sinistros. 

In [0]:
%sql
SELECT descricao AS REGIAO, AVG(numSinistros) AS MEDIA_SINISTROS
FROM SinReg
WHERE tipo_sin != 'TOTAL' --Tirando as somas de total de cada região
GROUP BY descricao
ORDER BY 2 DESC;


Questão: Utilize SQL para calcular a média de premio1 por sexo no arquivo arq_casco_comp. Como você interpretaria as diferenças encontradas entre os sexos?

O prêmio em média maior para o sexo masculino, sugere que as seguradoras entendem que o homem tem mais risco de se envolver em algum sinistro do que a mulher por diversos fatores analisados. 

In [0]:
%sql
SELECT SEXO, ROUND(AVG(try_cast(PREMIO1 as FLOAT)), 2) as MEDIA_PREMIO
FROM arq_casco_comp
GROUP BY SEXO
ORDER BY 2 DESC;

Questão: Como você consultaria a média de idade por ano do modelo no arquivo arq_casco_comp? Que conclusões podem ser tiradas sobre a relação entre a idade dos segurados e o ano do modelo dos veículos?

Mais uma vez a média de idade dos carros de até 20 anos é bem parecida, até que a média começa a ficar ligeiramente maior conforme mais antigo é o modelo. Há um pico na média de idade em carros extremamente antigos, que pertencem a pesssoas igualmente mais velhas por se tratar provavelmente de itens de coleção.

In [0]:
df = spark.sql(f"SELECT ANO_MODELO, round(AVG(IDADE), 3) AS MEDIA_IDADE\
              FROM arq_casco_comp\
              WHERE IDADE != 0\
              GROUP BY ANO_MODELO\
              ORDER BY 1 DESC\
              ")

df.display()

### Projetando a tabela arq_casco3_comp

In [0]:
df = spark.sql(f"SELECT * FROM arq_casco3_comp")

df.display()

### Projetando regiões com maior número de sinistros

In [0]:

df = spark.sql(f"SELECT regiao, descricao, numSinistros \
                FROM SinReg \
                WHERE tipo_sin = 'TOTAL' \
                ORDER BY numSinistros DESC \
                ")

df.display()

### Categorias de veículo com mais furtos

In [0]:
df = spark.sql(f"SELECT ac.CATEGORIA, SUM(arqc.FREQ_SIN1)\
               FROM arq_casco_comp arqc\
               INNER JOIN auto_cat ac\
               ON ac.CODIGO = arqc.COD_TARIF\
               GROUP BY ac.CATEGORIA\
               ORDER BY 2 DESC")

df.display()

Questão: Usando SQL, como você calcularia a média de idade dos segurados por sexo no arquivo arq_casco_comp? Qual é a interpretação desses resultados em termos de perfil dos segurados?

A grande maioria dos segurados são homens e acima dos 36 anos. 


In [0]:
df = spark.sql(f"SELECT acc.SEXO, ai.descricao, count(*) as QUANTIDADE\
               FROM arq_casco_comp acc\
               INNER JOIN auto_idade ai\
               ON acc.IDADE = ai.CODIGO\
               GROUP BY acc.SEXO, ai.descricao\
               ORDER BY 3 DESC\
               ")

df.display()

Questão: Escreva uma consulta SQL para calcular a média de idade por região no arquivo arq_casco_comp. O que esses resultados podem indicar sobre a distribuição demográfica dos segurados por região?


A média das idades é bem parecida em todos as regiões. É impreciso dizer alguma relação mais específica porque só temos as faixas de idade.


In [0]:
#Projetando o nome da região e média da idade com 2 casas decimais
df = spark.sql(f"SELECT ar.DESCRICAO, round(AVG(acc.IDADE), 3) AS MEDIA_IDADE\
               FROM arq_casco_comp acc\
               INNER JOIN auto_reg ar\
               ON try_cast(acc.REGIAO AS INT) = ar.CODIGO\
               WHERE IDADE != 0\
               GROUP BY ar.DESCRICAO\
               ORDER BY MEDIA_IDADE DESC--Ordenando por ordem decrescente\
               ")

df.display()